# Derive models of drone

Do all imports.

In [1]:
import sympy as sym
import numpy as np

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

## Dynamic model

Define physical parameters.

In [2]:
params = {
    'm': 0.5,
    'Jx': 0.0023,
    'Jy': 0.0023,
    'Jz': 0.0040,
    'l': 0.175,
    'g': 9.81,
}

Derive the equations of motion:

In [3]:
# components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z')

# yaw, pitch, roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
v_in_body = sym.Matrix([v_x, v_y, v_z])

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
w_in_body = sym.Matrix([w_x, w_y, w_z])

# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

# parameters
m = sym.nsimplify(params['m'])
Jx = sym.nsimplify(params['Jx'])
Jy = sym.nsimplify(params['Jy'])
Jz = sym.nsimplify(params['Jz'])
l = sym.nsimplify(params['l'])
g = sym.nsimplify(params['g'])
J = sym.diag(Jx, Jy, Jz)

# rotation matrices
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
R_body_in_world = Rz @ Ry @ Rx

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# applied forces
f_in_body = R_body_in_world.T @ sym.Matrix([[0], [0], [-m * g]]) + sym.Matrix([[0], [0], [f_z]])

# applied torques
tau_in_body = sym.Matrix([[tau_x], [tau_y], [tau_z]])

# equations of motion
f = sym.Matrix.vstack(
    R_body_in_world @ v_in_body,
    M @ w_in_body,
    (1 / m) * (f_in_body - w_in_body.cross(m * v_in_body)),
    J.inv() @ (tau_in_body - w_in_body.cross(J @ w_in_body)),
)

f = sym.simplify(f, full=True)

The equations of motion have this form:

$$\begin{bmatrix} \dot{p}_x \\ \dot{p}_y \\ \dot{p}_z \\ \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \\ \dot{v}_x \\ \dot{v}_y \\ \dot{v}_z \\ \dot{w}_x \\ \dot{w}_y \\ \dot{w}_z \end{bmatrix} = f\left(p_x, p_y, p_z, \psi, \theta, \phi, v_x, v_y, v_z, w_x, w_y, w_z, \tau_x, \tau_y, \tau_z, f_z \right)$$

Here is the function $f$:

In [4]:
f

Matrix([
[v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) - v_z*(sin(phi)*cos(psi) - sin(psi)*sin(theta)*cos(phi))],
[                                                                      -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                                 (w_y*sin(phi) + w_z*cos(phi))/cos(theta)],
[                                                                                                              w_y*cos(phi) - w_z*sin(phi)],
[                                                                                  w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                   v_y*w_z - v_z*w_y + 981*sin(theta)/100],
[   

## Sensor model

Define the sensor model.

In [5]:
# Position of drone in world frame
p_in_world = sym.Matrix([p_x, p_y, p_z])

# Position of markers in body frame
a_in_body = sym.Matrix([0, l, 0])  # <-- marker on left rotor
b_in_body = sym.Matrix([0, -l, 0]) # <-- marker on right rotor

# Position of markers in world frame
a_in_world = p_in_world + R_body_in_world @ a_in_body
b_in_world = p_in_world + R_body_in_world @ b_in_body

# Sensor model
g = sym.simplify(sym.Matrix.vstack(a_in_world, b_in_world))

The sensor model has this form:

$$o = g(p_x, p_y, p_z, \psi, \theta, \phi)$$

Here is the function $g$:

In [6]:
g

Matrix([
[p_x + 7*sin(phi)*sin(theta)*cos(psi)/40 - 7*sin(psi)*cos(phi)/40],
[p_y + 7*sin(phi)*sin(psi)*sin(theta)/40 + 7*cos(phi)*cos(psi)/40],
[                                  p_z + 7*sin(phi)*cos(theta)/40],
[p_x - 7*sin(phi)*sin(theta)*cos(psi)/40 + 7*sin(psi)*cos(phi)/40],
[p_y - 7*sin(phi)*sin(psi)*sin(theta)/40 - 7*cos(phi)*cos(psi)/40],
[                                  p_z - 7*sin(phi)*cos(theta)/40]])

## Find A and B

In [20]:
from scipy.linalg import solve_continuous_are
import numpy.linalg as la

m = (p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z)
n = (tau_x,tau_y,tau_z,f_z)
mn = (p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,tau_x,tau_y,tau_z,f_z)

An = sym.lambdify(mn, f.jacobian(m))
Bn = sym.lambdify(mn, f.jacobian(n))

A = An(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
B = Bn(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
print("Matrix A:")
print(A.tolist())
print("Matrix B:")
print(B.tolist())

Matrix A:
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, -9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
Matrix B:
[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 2.0], [434.7826086956522, 0.0, 0.0,

In [13]:
def lqr(A, B, Q, R):
    P = solve_continuous_are(A, B, Q, R)
    K = la.inv(R) @ B.T @ P
    return K

In [14]:
Q = np.diag((1,1,1,1,1,1,1,1,1,1,1,1)) * 1
R = np.diag((1,1,1,1)) * 1

W = B
for i in range(1, A.shape[0]):
    W = np.block([W, la.matrix_power(A, i) @ B])

if W.shape[0] == la.matrix_rank(W):
    print("System is controllable.")
    K = lqr(A,B,Q,R)
    print("Matrix K:")
    print(K.tolist())
else:
    print("System is not controllable.")

System is controllable.
Matrix K:
[[1.0690978283355357e-16, -0.9999999999999986, 9.216661203999703e-17, 1.9123884871291397e-15, 3.4970580298294006e-15, 5.462132179920029, -3.1471776083481423e-15, -1.4538172313903344, 3.869534644780097e-15, 1.0124849668156224, 1.1519001360851602e-16, 2.0420473967275664e-16], [1.0000000000000009, 1.6735294837481088e-15, 1.401165385307871e-16, 4.876321589103552e-16, 5.462132179920066, 5.743508558311876e-15, 1.453817231390347, 1.0367594817247018e-16, 2.833104167048431e-16, 1.1519001360851602e-16, 1.0124849668156217, 5.2095906539437894e-17], [1.7826550200871787e-16, -7.114706402618334e-17, -6.009871410775964e-16, 1.0000000000000013, 4.40194436770896e-15, 1.1375719984630585e-15, -1.4535406810881632e-15, -3.150453237707317e-16, -2.2454950149551625e-16, 1.1741772531183507e-16, 2.995514626017679e-17, 1.00399203184089], [1.612671570200053e-16, 2.5234993309509705e-16, 0.9999999999999996, -6.609906583286819e-17, 4.1744919651083055e-16, -1.0211337709790963e-15, 1.7

In [15]:
o_e = g.subs({p_x: 0, p_y: 0, p_z: 0, psi: 0, theta: 0, phi: 0})
print(o_e.evalf().tolist())

[[0], [0.175000000000000], [0], [0], [-0.175000000000000], [0]]


In [16]:
m = (p_x,p_y,p_z,psi,theta,phi)
n = ()
mn = (p_x,p_y,p_z,psi,theta,phi)

# No D because there are no inputs in the sensor model
Cn = sym.lambdify(mn, g.jacobian(m))
#Dn = sym.lambdify(mn, f.jacobian(n))

C = Cn(0,0,0,0,0,0)
C = np.hstack((C, np.zeros((C.shape[0], 6))))
#D = Dn(0,0,0,0,0,0)
print("Matrix C:")
print(C.tolist())
# print("Matrix D:")
# print(D.tolist())

Matrix C:
[[1.0, 0.0, 0.0, -0.175, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, -0.0, 0.175, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.175, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, -0.175, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]


In [18]:
# Define weight matrices for LQR
Q0 = np.eye(C.shape[0]) * 1
R0 = np.eye(A.shape[0]) * 1

O = C
for i in range(1, A.shape[0]):
    O = np.vstack([O, C @ la.matrix_power(A, i)])

if A.shape[0] == la.matrix_rank(O):
    print("System is observable.")
    L = lqr(A.T, C.T, la.inv(R0), la.inv(Q0)).T
    print("Matrix L:")
    print(L.tolist())
else:
    print("System is not observable, the rank is", la.matrix_rank(O))

System is observable.
Matrix L:
[[2.9753760189621703, 1.0081449585351537e-15, -9.168280250544893e-17, 2.9753760189621703, 1.0081449585351537e-15, 9.16828025001474e-17], [1.013529909750936e-15, 2.9271137918024235, -0.21976816368497756, 1.0027600073193713e-15, 2.9271137918024235, 0.21976816368497815], [2.3735251220963706e-18, 2.948667518607218e-16, 1.0986841134678091, -2.373525127397908e-18, 2.948667518607218e-16, 1.0986841134678091], [-2.130870755866928, -3.0771149804471156e-17, -8.063485983032896e-17, 2.1308707558669253, -3.0771149804471156e-17, 5.350885840464738e-17], [1.3054317390112538, 1.0248304349207444e-15, -8.449765858237292e-17, 1.3054317390112533, 1.0248304349207444e-15, 8.449765857836918e-17], [-4.568298694699299e-16, -1.2558180781998736, 0.3042973803557072, -5.909735877049063e-16, -1.2558180781998736, -0.3042973803557077], [8.352862454215185, 4.6236819209162666e-15, -4.055583426375225e-16, 8.352862454215185, 4.6236819209162666e-15, 4.0555834261897337e-16], [2.980429248305942